In [ ]:
# Setup
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

files_objs = os.listdir(os.path.join("C:\\Users\CLO\OneDrive - Stryker\\School\\Qualcomm Hackathon", "cards_image"))
files_objs = [os.path.join("C:\\Users\\CLO\\OneDrive - Stryker\\School\\Qualcomm Hackathon", "cards_image", f) for f in files_objs]

def output_images(similar, target, n=5):
    display_img(target, "original")
    counter = 0
    for k, v in similar.items():
        display_img(k, "similarity:" + str(v))
        counter += 1
        if counter == n:
            break

    return

def display_img(path, title):
    plt.imshow(Image.open(path))
    plt.axis("off")
    plt.title(title)
    plt.show()

In [ ]:
# Load model and get embeddings from dataset

# Load model (e.g., ResNet18)
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the classification head
model.eval()

# define preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Get embeddings
dataset = {}
for file in files_objs:
    image = Image.open(file)
    # Check if the image has an alpha channel (RGBA)
    if image.mode == 'RGBA' or image.mode == 'P':
        # Remove the alpha channel by converting to RGB
        image = image.convert('RGB')
    print(file)
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        vector = model(input_tensor).flatten()
    dataset[str(file)] = vector

data = {"model": "resnet18", "embeddings": dataset}
torch.save(
    data, "resnet18_embeddings.pt"
)  # need to update functionality for naming convention



In [ ]:
# Load model to get image embeddings
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the classification head
model.eval()

# define preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

target = "test_embedding3.jpg"
image = Image.open(target)
input_tensor = transform(image).unsqueeze(0)
with torch.no_grad():
    target_vector = model(input_tensor).flatten().unsqueeze(0)

n = None
# initiate computation of consine similarity
cosine = nn.CosineSimilarity(dim=1)

# iteratively store similarity of stored images to target image
sim_dict = {}
for k, v in dataset.items():
    sim = cosine(v.unsqueeze(0), target_vector)[0].item()
    sim_dict[k] = sim

# sort based on decreasing similarity
items = sim_dict.items()
sim_dict = {k: v for k, v in sorted(items, key=lambda i: i[1], reverse=True)}

# cut to defined top n similar images
if n is not None:
    sim_dict = dict(list(sim_dict.items())[: int(n)])

output_images(sim_dict, target)